In [ ]:
!pip3 install vaderSentiment
!pip3 install flair
!pip3 install spacytextblob

In [ ]:
!python3 -m spacy download en_core_web_sm

     |████████████████████████████████| 13.6 MB 70 kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive


In [ ]:
cd MyDrive/Omdena\ Colombia\ Chapter

/gdrive/MyDrive/Omdena Colombia Chapter


In [ ]:
import pandas as pd
import re
import string
from google.colab import files
import ast

In [ ]:
data = pd.read_csv("Corrected_Final_All.csv")

In [ ]:
data_english = data[data['lang']=='en']

In [ ]:
data_english.isna().sum()

Unnamed: 0                               0
created_at                               0
id_str                                1039
conversation_id_str                   1039
full_text                                0
twitter_lang                          1039
favorited                             1039
retweeted                             1039
retweet_count                         1039
favorite_count                        1039
reply_count                           1039
quote_count                           1039
quoted_status_id_str                  4005
quoted_status_short_url               5470
quoted_status_expand_url              5470
user_id_str                           1039
user_name                             1039
user_full_name                        1040
user_verified                         1039
in_reply_to_status_id_str             4445
in_reply_to_user_id_str               4254
hashtags                              1040
mentions                              3351
urls       

In [ ]:
#To preprocess list present as string, specially for column "preprocessed_tweets_without_hashtags"
import ast
x = '[ "I","am","not","here"]'
x  = ast.literal_eval(x)
print(x) ##returns as list of strings ['I', 'am', 'not', 'here']
print(" ".join(x)) ##returns as concatenated string separated by spaces --> I am not here

['I', 'am', 'not', 'here']
I am not here


change name of column according to what column you are using for sentiment analysis

In [ ]:
#TextBlob
from textblob import TextBlob

score_textBlob = []

for i in range(data_english.shape[0]):
  testimonial = TextBlob(data['full_text'].iloc[i])
  pol = testimonial.polarity
  if pol >= 0:
    score_textBlob.append("pos")
  else:
    score_textBlob.append("neg")

data_english['sentiment_full_text_textBlob'] = score_textBlob

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
#VADER
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

analyzer = SentimentIntensityAnalyzer()
score_VADER = []

for i in range(data_english.shape[0]):
  vs = analyzer.polarity_scores(data['full_text'].iloc[i])
  comp = vs['compound']
  if comp >= 0:
    score_VADER.append("pos")
  else:
    score_VADER.append("neg")

data_english['sentiment_full_text_vader'] = score_VADER

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


In [ ]:
#Flair
from flair.models import TextClassifier
from flair.data import Sentence

classifier = TextClassifier.load('en-sentiment')
score_Flair = []

for i in range(data_english.shape[0]):
  sentence = Sentence(data['full_text'].iloc[i])
  classifier.predict(sentence)
  comp = sentence.labels

  if "POSITIVE" in str(comp):
    score_Flair.append("pos")
  else:
    score_Flair.append("neg")

data_english['sentiment_full_text_flair'] = score_Flair

2021-09-25 08:20:54,623 loading file /root/.flair/models/sentiment-en-mix-distillbert_4.pt


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
#SpacyTextBlob
import spacy
from spacytextblob.spacytextblob import SpacyTextBlob

nlp = spacy.load('en_core_web_sm')
nlp.add_pipe('spacytextblob')
score_spacy = []

for i in range(data_english.shape[0]):
  text = data['full_text'].iloc[i]
  doc = nlp(text)
  pol = doc._.polarity
  if pol >= 0:
    score_spacy.append("pos")
  else:
    score_spacy.append("neg")

data_english['sentiment_full_text_spacytextblob'] = score_spacy

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
data_english.tail(5)

,Unnamed: 0,created_at,id_str,conversation_id_str,full_text,twitter_lang,favorited,retweeted,retweet_count,favorite_count,reply_count,quote_count,quoted_status_id_str,quoted_status_short_url,quoted_status_expand_url,user_id_str,user_name,user_full_name,user_verified,in_reply_to_status_id_str,in_reply_to_user_id_str,hashtags,mentions,urls,media,preprocessed_created_at,preprocessed_data,emoji_list,emoticons_list,filename,data_source,lang,score,langTb,lang_langdetect,preprocessed_data_without_hashtags,sentiment_full_text_textBlob,sentiment_full_text_vader,sentiment_full_text_flair,sentiment_full_text_spacytextblob
22155,22155,2021-08-03 12:01:04,NaN,NaN,It looks like you shared an AMP link. These sh...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-08-03,"['it', 'looks', 'like', 'you', 'shared', 'an',...",[''],['d:'],NaN,Reddit,en,2.0,NaN,en,"['look', 'like', 'share', 'amp', 'link', 'load...",pos,pos,neg,pos
22156,22156,2021-08-05 22:02:30,NaN,NaN,Su post o comentario ha sido removido por viol...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-08-05,"['post', 'comentario', 'ser', 'remover', 'viol...",[''],[],NaN,Reddit,en,1.0,NaN,en,"['su', 'post', 'comentario', 'ha', 'sido', 're...",pos,pos,neg,pos
22157,22157,2021-08-06 17:19:38,NaN,NaN,Su post o comentario ha sido removido por viol...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-08-06,"['post', 'comentario', 'ser', 'remover', 'viol...",[''],[],NaN,Reddit,en,1.0,NaN,en,"['su', 'post', 'comentario', 'ha', 'sido', 're...",pos,pos,neg,pos
22158,22158,2021-08-11 23:03:45,NaN,NaN,It looks like you shared an AMP link. These sh...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-08-11,"['it', 'looks', 'like', 'you', 'shared', 'an',...",[''],['d:'],NaN,Reddit,en,2.0,NaN,en,"['look', 'like', 'share', 'amp', 'link', 'load...",pos,pos,neg,pos
22159,22159,2021-08-13 6:01:44,NaN,NaN,you can't troll the troll broh sorry.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-08-13,"['you', 'cannot', 'troll', 'the', 'troll', 'br...",[''],[],NaN,Reddit,en,3.0,NaN,en,"['cannot', 'troll', 'troll', 'broh', 'sorry']",pos,pos,neg,pos


In [ ]:
#selecting columns
df = data_english[['sentiment_full_text_textBlob', 'sentiment_full_text_vader', 'sentiment_full_text_spacytextblob', 'sentiment_full_text_flair']]
df


,sentiment_full_text_textBlob,sentiment_full_text_vader,sentiment_full_text_spacytextblob,sentiment_full_text_flair
16690,pos,pos,pos,neg
16691,pos,pos,pos,neg
16692,pos,pos,pos,neg
16693,pos,neg,pos,neg
16694,pos,pos,pos,neg
...,...,...,...,...
22155,pos,pos,pos,neg
22156,pos,pos,pos,neg
22157,pos,pos,pos,neg
22158,pos,pos,pos,neg


In [ ]:
#finding the mode/majority
df['majority'] = df.mode(axis=1)[0]
df

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,sentiment_full_text_textBlob,sentiment_full_text_vader,sentiment_full_text_spacytextblob,sentiment_full_text_flair,majority
16690,pos,pos,pos,neg,pos
16691,pos,pos,pos,neg,pos
16692,pos,pos,pos,neg,pos
16693,pos,neg,pos,neg,neg
16694,pos,pos,pos,neg,pos
...,...,...,...,...,...
22155,pos,pos,pos,neg,pos
22156,pos,pos,pos,neg,pos
22157,pos,pos,pos,neg,pos
22158,pos,pos,pos,neg,pos


In [ ]:
#dropping columns
df = df.drop(['sentiment_full_text_textBlob', 'sentiment_full_text_vader', 'sentiment_full_text_spacytextblob', 'sentiment_full_text_flair'], axis = 1)

In [ ]:
#concatinating two dataframe
df_sentiment = pd.concat([data_english, df], axis=1)
df_sentiment

,Unnamed: 0,created_at,id_str,conversation_id_str,full_text,twitter_lang,favorited,retweeted,retweet_count,favorite_count,reply_count,quote_count,quoted_status_id_str,quoted_status_short_url,quoted_status_expand_url,user_id_str,user_name,user_full_name,user_verified,in_reply_to_status_id_str,in_reply_to_user_id_str,hashtags,mentions,urls,media,preprocessed_created_at,preprocessed_data,emoji_list,emoticons_list,filename,data_source,lang,score,langTb,lang_langdetect,preprocessed_data_without_hashtags,sentiment_full_text_textBlob,sentiment_full_text_vader,sentiment_full_text_flair,sentiment_full_text_spacytextblob,majority
16690,16690,2021-04-23T07:14:36+00:00,1.390000e+18,1.390000e+18,📢Join the seminar aiming at supporting #1May #...,en,False,False,5.0,4.0,0.0,1.0,NaN,NaN,NaN,1.050000e+18,_EuropeanForum,European Forum,False,NaN,NaN,"#1May , #1maggio , #1Mayo","LeftHeinz , Yolanda_Diaz_ , VisentiniLuca , lu...",NaN,"[{""url"": ""https://pbs.twimg.com/media/EzpAFBfX...",2021-04-23,"['join', 'seminar', 'aim', 'support', 'may', '...",['📢'],[],Mayo_ENGLISH_tweets_stweet.csv,Twitter,en,NaN,NaN,NaN,"['join', 'seminar', 'aim', 'support', 'trade',...",pos,pos,neg,pos,pos
16691,16691,2021-04-22T04:12:43+00:00,1.390000e+18,1.380000e+18,@julioacosta1701 @DiazCanelB @AlejandroGilF @r...,es,False,False,7.0,1.0,0.0,0.0,NaN,NaN,NaN,1.210000e+18,BauzaCuba,Alejandro Rodríguez Bauza,False,1.380000e+18,1.190000e+18,"#PrevenirPorLaVida , #PatriaOMuerte , #UnBlock...","julioacosta1701 , DiazCanelB , AlejandroGilF ,...",NaN,[],2021-04-22,"['caliente', 'tweetazo', 'prevenirporlavida', ...",[],[],Mayo_ENGLISH_tweets_stweet.csv,Twitter,en,NaN,NaN,NaN,"['caliente', 'tweetazo']",pos,pos,neg,pos,pos
16692,16692,2021-04-22T04:12:00+00:00,1.390000e+18,1.380000e+18,@julioacosta1701 @AliRubioGlez @Vicente7397772...,es,False,False,7.0,1.0,0.0,0.0,NaN,NaN,NaN,1.210000e+18,BauzaCuba,Alejandro Rodríguez Bauza,False,1.380000e+18,1.190000e+18,"#PrevenirPorLaVida , #PatriaOMuerte , #UnBlock...","julioacosta1701 , AliRubioGlez , Vicente739777...",NaN,[],2021-04-22,"['sumo', 'prevenirporlavida', 'patriaomuerte',...",[],[],Mayo_ENGLISH_tweets_stweet.csv,Twitter,en,NaN,NaN,NaN,['sumo'],pos,pos,neg,pos,pos
16693,16693,2021-04-22T04:09:48+00:00,1.390000e+18,1.380000e+18,@AdrinMartnezGo1 @julioacosta1701 @DiazCanelB ...,es,False,False,6.0,0.0,0.0,0.0,NaN,NaN,NaN,1.210000e+18,BauzaCuba,Alejandro Rodríguez Bauza,False,1.390000e+18,1.140000e+18,"#PrevenirPorLaVida , #PatriaOMuerte , #UnBlock...","AdrinMartnezGo1 , julioacosta1701 , DiazCanelB...",NaN,[],2021-04-22,"['hay', 'diocancancan', 'prevenirporlavida', '...",['😅😂🤣🤣'],[],Mayo_ENGLISH_tweets_stweet.csv,Twitter,en,NaN,NaN,NaN,"['hay', 'diocancancan']",pos,neg,neg,pos,neg
16694,16694,2021-04-22T03:32:26+00:00,1.390000e+18,1.380000e+18,@VilmaPrez14 @Laura53538740 @Granma_Digital @D...,es,False,False,5.0,1.0,0.0,1.0,NaN,NaN,NaN,1.210000e+18,BauzaCuba,Alejandro Rodríguez Bauza,False,1.380000e+18,1.100000e+18,"#PrevenirPorLaVida , #PatriaOMuerte , #UnBlock...","VilmaPrez14 , Laura53538740 , Granma_Digital ,...",NaN,[],2021-04-22,"['con', 'mucha', 'conciencia', 'prevenirporlav...",[],[],Mayo_ENGLISH_tweets_stweet.csv,Twitter,en,NaN,NaN,NaN,"['con', 'mucha', 'conciencia']",pos,pos,neg,pos,pos
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22155,22155,2021-08-03 12:01:04,NaN,NaN,It looks like you shared an AMP link. These sh...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-08-03,"['it', 'looks', 'like', 'you', 'shared', 'an',...",[''],['d:'],NaN,Reddit,en,2.0,NaN,en,"['look', 'like', 'share', 'amp', 'link', 'load...",pos,pos,neg,pos,pos
22156,22156,2021-08-05 22:02:30,NaN,NaN,Su post o comentario ha sido removido por viol...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-08-05,"['post', 'comentario', 'ser', 'remover', 'viol...",[''],[],N

In [ ]:
#changing string to integer
score = []
for i in range(df_sentiment.shape[0]):
  text = df_sentiment['majority'].iloc[i]
  if text == 'pos':
    score.append('1')
  else:
    score.append('0')
df_sentiment['sentiment_score_full_text'] = score

In [ ]:
#Download file with the name "Corrected_Final_All_English_[column_name].csv"
df_sentiment.to_csv("Corrected_Final_All_English_Full_Text.csv", index=False)
files.download("Corrected_Final_All_English_Full_Text.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
data.columns

Index(['Unnamed: 0', 'created_at', 'id_str', 'conversation_id_str',
       'full_text', 'twitter_lang', 'favorited', 'retweeted', 'retweet_count',
       'favorite_count', 'reply_count', 'quote_count', 'quoted_status_id_str',
       'quoted_status_short_url', 'quoted_status_expand_url', 'user_id_str',
       'user_name', 'user_full_name', 'user_verified',
       'in_reply_to_status_id_str', 'in_reply_to_user_id_str', 'hashtags',
       'mentions', 'urls', 'media', 'preprocessed_created_at',
       'preprocessed_data', 'emoji_list', 'emoticons_list', 'filename',
       'data_source', 'lang', 'score', 'langTb', 'lang_langdetect',
       'preprocessed_data_without_hashtags'],
      dtype='object')

['join', 'seminar', 'aim', 'support', 'trade', 'union', 'activity', 'amp', 'mobilization', '📢']


In [ ]:
list_text_emoji = []
for i in range(data_english.shape[0]):
  x = ast.literal_eval(data_english['preprocessed_data_without_hashtags'].values[i])
  y = ast.literal_eval(data_english['emoji_list'].values[i])
  z = ast.literal_eval(data_english['emoticons_list'].values[i])
  list_text_emoji.append(x+y+z)

data_english['text_emoji_emoticon_combined'] = list_text_emoji

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
data_english.shape

(5470, 37)

In [ ]:
#TextBlob
from textblob import TextBlob

score_textBlob = []

for i in range(data_english.shape[0]):
  sentence = " ".join(data_english['text_emoji_emoticon_combined'].iloc[i])
  testimonial = TextBlob(sentence)
  pol = testimonial.polarity
  if pol >= 0:
    score_textBlob.append("pos")
  else:
    score_textBlob.append("neg")

data_english['sentiment_text_emoji_emoticon_combined_textBlob'] = score_textBlob

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


In [ ]:
#VADER
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

analyzer = SentimentIntensityAnalyzer()
score_VADER = []

for i in range(data_english.shape[0]):
  sentence = " ".join(data_english['text_emoji_emoticon_combined'].iloc[i])
  vs = analyzer.polarity_scores(sentence)
  comp = vs['compound']
  if comp >= 0:
    score_VADER.append("pos")
  else:
    score_VADER.append("neg")

data_english['sentiment_text_emoji_emoticon_combined_vader'] = score_VADER

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


In [ ]:
#Flair
from flair.models import TextClassifier
from flair.data import Sentence

classifier = TextClassifier.load('en-sentiment')
score_Flair = []

for i in range(data_english.shape[0]):
  sentence = Sentence(" ".join(data_english['text_emoji_emoticon_combined'].iloc[i]))
  classifier.predict(sentence)
  comp = sentence.labels
  if comp == "POS":
    score_Flair.append("pos")
  else:
    score_Flair.append("neg")

data_english['sentiment_text_emoji_emoticon_combined_flair'] = score_Flair

2021-09-25 06:35:47,680 loading file /root/.flair/models/sentiment-en-mix-distillbert_4.pt
2021-09-25 06:36:01,721 Warning: An empty Sentence was created! Are there empty strings in your dataset?
2021-09-25 06:36:19,940 Warning: An empty Sentence was created! Are there empty strings in your dataset?
2021-09-25 06:36:32,823 Warning: An empty Sentence was created! Are there empty strings in your dataset?
2021-09-25 06:36:42,303 Warning: An empty Sentence was created! Are there empty strings in your dataset?
2021-09-25 06:36:53,320 Warning: An empty Sentence was created! Are there empty strings in your dataset?
2021-09-25 06:36:53,366 Warning: An empty Sentence was created! Are there empty strings in your dataset?
2021-09-25 06:36:55,996 Warning: An empty Sentence was created! Are there empty strings in your dataset?
2021-09-25 06:36:56,084 Warning: An empty Sentence was created! Are there empty strings in your dataset?
2021-09-25 06:36:56,085 Warning: An empty Sentence was created! Are t

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
#SpacyTextBlob
import spacy
from spacytextblob.spacytextblob import SpacyTextBlob

nlp = spacy.load('en_core_web_sm')
nlp.add_pipe('spacytextblob')
score_spacy = []

for i in range(data_english.shape[0]):
  text = " ".join(data_english['text_emoji_emoticon_combined'].iloc[i])
  doc = nlp(text)
  pol = doc._.polarity
  if pol >= 0:
    score_spacy.append("pos")
  else:
    score_spacy.append("neg")

data_english['sentiment_text_emoji_emoticon_combined_spacytextblob'] = score_spacy

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
data_english.tail(5)

,Unnamed: 0,created_at,id_str,conversation_id_str,full_text,twitter_lang,favorited,retweeted,retweet_count,favorite_count,reply_count,quote_count,quoted_status_id_str,quoted_status_short_url,quoted_status_expand_url,user_id_str,user_name,user_full_name,user_verified,in_reply_to_status_id_str,in_reply_to_user_id_str,hashtags,mentions,urls,media,preprocessed_created_at,preprocessed_data,emoji_list,emoticons_list,filename,data_source,lang,score,langTb,lang_langdetect,preprocessed_data_without_hashtags,text_emoji_emoticon_combined,sentiment_text_emoji_emoticon_combined_textBlob,sentiment_text_emoji_emoticon_combined_vader,sentiment_text_emoji_emoticon_combined_flair,sentiment_text_emoji_emoticon_combined_spacytextblob
22155,22155,2021-08-03 12:01:04,NaN,NaN,It looks like you shared an AMP link. These sh...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-08-03,"['it', 'looks', 'like', 'you', 'shared', 'an',...",[''],['d:'],NaN,Reddit,en,2.0,NaN,en,"['look', 'like', 'share', 'amp', 'link', 'load...","[look, like, share, amp, link, load, faster, g...",pos,pos,neg,pos
22156,22156,2021-08-05 22:02:30,NaN,NaN,Su post o comentario ha sido removido por viol...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-08-05,"['post', 'comentario', 'ser', 'remover', 'viol...",[''],[],NaN,Reddit,en,1.0,NaN,en,"['su', 'post', 'comentario', 'ha', 'sido', 're...","[su, post, comentario, ha, sido, removido, por...",neg,neg,neg,neg
22157,22157,2021-08-06 17:19:38,NaN,NaN,Su post o comentario ha sido removido por viol...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-08-06,"['post', 'comentario', 'ser', 'remover', 'viol...",[''],[],NaN,Reddit,en,1.0,NaN,en,"['su', 'post', 'comentario', 'ha', 'sido', 're...","[su, post, comentario, ha, sido, removido, por...",neg,neg,neg,neg
22158,22158,2021-08-11 23:03:45,NaN,NaN,It looks like you shared an AMP link. These sh...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-08-11,"['it', 'looks', 'like', 'you', 'shared', 'an',...",[''],['d:'],NaN,Reddit,en,2.0,NaN,en,"['look', 'like', 'share', 'amp', 'link', 'load...","[look, like, share, amp, link, load, faster, g...",pos,pos,neg,pos
22159,22159,2021-08-13 6:01:44,NaN,NaN,you can't troll the troll broh sorry.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-08-13,"['you', 'cannot', 'troll', 'the', 'troll', 'br...",[''],[],NaN,Reddit,en,3.0,NaN,en,"['cannot', 'troll', 'troll', 'broh', 'sorry']","[cannot, troll, troll, broh, sorry, ]",neg,neg,neg,neg


In [ ]:
df = data_english[['sentiment_text_emoji_emoticon_combined_textBlob', 'sentiment_text_emoji_emoticon_combined_vader', 
                   'sentiment_text_emoji_emoticon_combined_spacytextblob', 'sentiment_text_emoji_emoticon_combined_flair']]

#finding the mode/majority
df['majority'] = df.mode(axis=1)[0]

#dropping columns
df = df.drop(['sentiment_text_emoji_emoticon_combined_textBlob', 'sentiment_text_emoji_emoticon_combined_vader', 
              'sentiment_text_emoji_emoticon_combined_spacytextblob', 'sentiment_text_emoji_emoticon_combined_flair'], axis = 1)

#concatinating two dataframe
df_sentiment = pd.concat([data_english, df], axis=1)
df_sentiment

#changing string to integer
score = []
for i in range(df_sentiment.shape[0]):
  text = df_sentiment['majority'].iloc[i]
  if text == 'pos':
    score.append('1')
  else:
    score.append('0')
df_sentiment['sentiment_score_text_emoji_emoticon_combined'] = score

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [ ]:
df_sentiment

,Unnamed: 0,created_at,id_str,conversation_id_str,full_text,twitter_lang,favorited,retweeted,retweet_count,favorite_count,reply_count,quote_count,quoted_status_id_str,quoted_status_short_url,quoted_status_expand_url,user_id_str,user_name,user_full_name,user_verified,in_reply_to_status_id_str,in_reply_to_user_id_str,hashtags,mentions,urls,media,preprocessed_created_at,preprocessed_data,emoji_list,emoticons_list,filename,data_source,lang,score,langTb,lang_langdetect,preprocessed_data_without_hashtags,text_emoji_emoticon_combined,sentiment_text_emoji_emoticon_combined_textBlob,sentiment_text_emoji_emoticon_combined_vader,sentiment_text_emoji_emoticon_combined_flair,sentiment_text_emoji_emoticon_combined_spacytextblob,majority,sentiment_score_text_emoji_emoticon_combined
16690,16690,2021-04-23T07:14:36+00:00,1.390000e+18,1.390000e+18,📢Join the seminar aiming at supporting #1May #...,en,False,False,5.0,4.0,0.0,1.0,NaN,NaN,NaN,1.050000e+18,_EuropeanForum,European Forum,False,NaN,NaN,"#1May , #1maggio , #1Mayo","LeftHeinz , Yolanda_Diaz_ , VisentiniLuca , lu...",NaN,"[{""url"": ""https://pbs.twimg.com/media/EzpAFBfX...",2021-04-23,"['join', 'seminar', 'aim', 'support', 'may', '...",['📢'],[],Mayo_ENGLISH_tweets_stweet.csv,Twitter,en,NaN,NaN,NaN,"['join', 'seminar', 'aim', 'support', 'trade',...","[join, seminar, aim, support, trade, union, ac...",pos,pos,neg,pos,pos,1
16691,16691,2021-04-22T04:12:43+00:00,1.390000e+18,1.380000e+18,@julioacosta1701 @DiazCanelB @AlejandroGilF @r...,es,False,False,7.0,1.0,0.0,0.0,NaN,NaN,NaN,1.210000e+18,BauzaCuba,Alejandro Rodríguez Bauza,False,1.380000e+18,1.190000e+18,"#PrevenirPorLaVida , #PatriaOMuerte , #UnBlock...","julioacosta1701 , DiazCanelB , AlejandroGilF ,...",NaN,[],2021-04-22,"['caliente', 'tweetazo', 'prevenirporlavida', ...",[],[],Mayo_ENGLISH_tweets_stweet.csv,Twitter,en,NaN,NaN,NaN,"['caliente', 'tweetazo']","[caliente, tweetazo]",pos,pos,neg,pos,pos,1
16692,16692,2021-04-22T04:12:00+00:00,1.390000e+18,1.380000e+18,@julioacosta1701 @AliRubioGlez @Vicente7397772...,es,False,False,7.0,1.0,0.0,0.0,NaN,NaN,NaN,1.210000e+18,BauzaCuba,Alejandro Rodríguez Bauza,False,1.380000e+18,1.190000e+18,"#PrevenirPorLaVida , #PatriaOMuerte , #UnBlock...","julioacosta1701 , AliRubioGlez , Vicente739777...",NaN,[],2021-04-22,"['sumo', 'prevenirporlavida', 'patriaomuerte',...",[],[],Mayo_ENGLISH_tweets_stweet.csv,Twitter,en,NaN,NaN,NaN,['sumo'],[sumo],pos,pos,neg,pos,pos,1
16693,16693,2021-04-22T04:09:48+00:00,1.390000e+18,1.380000e+18,@AdrinMartnezGo1 @julioacosta1701 @DiazCanelB ...,es,False,False,6.0,0.0,0.0,0.0,NaN,NaN,NaN,1.210000e+18,BauzaCuba,Alejandro Rodríguez Bauza,False,1.390000e+18,1.140000e+18,"#PrevenirPorLaVida , #PatriaOMuerte , #UnBlock...","AdrinMartnezGo1 , julioacosta1701 , DiazCanelB...",NaN,[],2021-04-22,"['hay', 'diocancancan', 'prevenirporlavida', '...",['😅😂🤣🤣'],[],Mayo_ENGLISH_tweets_stweet.csv,Twitter,en,NaN,NaN,NaN,"['hay', 'diocancancan']","[hay, diocancancan, 😅😂🤣🤣]",pos,pos,neg,pos,pos,1
16694,16694,2021-04-22T03:32:26+00:00,1.390000e+18,1.380000e+18,@VilmaPrez14 @Laura53538740 @Granma_Digital @D...,es,False,False,5.0,1.0,0.0,1.0,NaN,NaN,NaN,1.210000e+18,BauzaCuba,Alejandro Rodríguez Bauza,False,1.380000e+18,1.100000e+18,"#PrevenirPorLaVida , #PatriaOMuerte , #UnBlock...","VilmaPrez14 , Laura53538740 , Granma_Digital ,...",NaN,[],2021-04-22,"['con', 'mucha', 'conciencia', 'prevenirporlav...",[],[],Mayo_ENGLISH_tweets_stweet.csv,Twitter,en,NaN,NaN,NaN,"['con', 'mucha', 'conciencia']","[con, mucha, conciencia]",pos,pos,neg,pos,pos,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22155,22155,2021-08-03 12:01:04,NaN,NaN,It looks like you shared an AMP link. These sh...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-08-03,"['it', 'looks', 'like', 'you', 'shared', 'an',...",[''],['d:'],NaN,Reddit,en,2.0,NaN,en,"['look',

In [ ]:
#Download file with the name "Corrected_Final_All_English_[column_name].csv"
df_sentiment.to_csv("Corrected_Final_All_English_Text_Emoji_Emoticon_Combined.csv", index=False)
files.download("Corrected_Final_All_English_Text_Emoji_Emoticon_Combined.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#TextBlob
from textblob import TextBlob

score_textBlob = []

for i in range(data_english.shape[0]):
  x = ast.literal_eval(data_english['preprocessed_data_without_hashtags'].values[i])
  testimonial = TextBlob(" ".join(x))
  pol = testimonial.polarity
  if pol >= 0:
    score_textBlob.append("pos")
  else:
    score_textBlob.append("neg")

data_english['sentiment_preprocessed_data_without_hashtags_textBlob'] = score_textBlob

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


In [ ]:
#VADER
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

analyzer = SentimentIntensityAnalyzer()
score_VADER = []

for i in range(data_english.shape[0]):
  x = ast.literal_eval(data_english['preprocessed_data_without_hashtags'].values[i])
  vs = analyzer.polarity_scores(" ".join(x))
  comp = vs['compound']
  if comp >= 0:
    score_VADER.append("pos")
  else:
    score_VADER.append("neg")

data_english['sentiment_preprocessed_data_without_hashtags_vader'] = score_VADER

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


In [ ]:
#Flair
from flair.models import TextClassifier
from flair.data import Sentence

classifier = TextClassifier.load('en-sentiment')
score_Flair = []

for i in range(data_english.shape[0]):
  x = ast.literal_eval(data_english['preprocessed_data_without_hashtags'].values[i])
  sentence = Sentence(" ".join(x))
  classifier.predict(sentence)
  comp = sentence.labels
  if comp == "POS":
    score_Flair.append("pos")
  else:
    score_Flair.append("neg")

data_english['sentiment_preprocessed_data_without_hashtags_flair'] = score_Flair

2021-09-25 08:04:17,322 loading file /root/.flair/models/sentiment-en-mix-distillbert_4.pt
2021-09-25 08:04:31,700 Warning: An empty Sentence was created! Are there empty strings in your dataset?
2021-09-25 08:04:49,989 Warning: An empty Sentence was created! Are there empty strings in your dataset?
2021-09-25 08:05:03,132 Warning: An empty Sentence was created! Are there empty strings in your dataset?
2021-09-25 08:05:12,661 Warning: An empty Sentence was created! Are there empty strings in your dataset?
2021-09-25 08:05:23,479 Warning: An empty Sentence was created! Are there empty strings in your dataset?
2021-09-25 08:05:23,523 Warning: An empty Sentence was created! Are there empty strings in your dataset?
2021-09-25 08:05:26,162 Warning: An empty Sentence was created! Are there empty strings in your dataset?
2021-09-25 08:05:26,253 Warning: An empty Sentence was created! Are there empty strings in your dataset?
2021-09-25 08:05:26,255 Warning: An empty Sentence was created! Are t

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
#SpacyTextBlob
import spacy
from spacytextblob.spacytextblob import SpacyTextBlob

nlp = spacy.load('en_core_web_sm')
nlp.add_pipe('spacytextblob')
score_spacy = []

for i in range(data_english.shape[0]):
  x = ast.literal_eval(data_english['preprocessed_data_without_hashtags'].values[i])
  text = " ".join(x)
  doc = nlp(text)
  pol = doc._.polarity
  if pol >= 0:
    score_spacy.append("pos")
  else:
    score_spacy.append("neg")

data_english['sentiment_preprocessed_data_without_hashtags_spacytextblob'] = score_spacy

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
data_english.tail(5)

,Unnamed: 0,created_at,id_str,conversation_id_str,full_text,twitter_lang,favorited,retweeted,retweet_count,favorite_count,reply_count,quote_count,quoted_status_id_str,quoted_status_short_url,quoted_status_expand_url,user_id_str,user_name,user_full_name,user_verified,in_reply_to_status_id_str,in_reply_to_user_id_str,hashtags,mentions,urls,media,preprocessed_created_at,preprocessed_data,emoji_list,emoticons_list,filename,data_source,lang,score,langTb,lang_langdetect,preprocessed_data_without_hashtags,sentiment_preprocessed_data_without_hashtags_textBlob,sentiment_preprocessed_data_without_hashtags_vader,sentiment_preprocessed_data_without_hashtags_flair,sentiment_preprocessed_data_without_hashtags_spacytextblob
22155,22155,2021-08-03 12:01:04,NaN,NaN,It looks like you shared an AMP link. These sh...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-08-03,"['it', 'looks', 'like', 'you', 'shared', 'an',...",[''],['d:'],NaN,Reddit,en,2.0,NaN,en,"['look', 'like', 'share', 'amp', 'link', 'load...",pos,pos,neg,pos
22156,22156,2021-08-05 22:02:30,NaN,NaN,Su post o comentario ha sido removido por viol...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-08-05,"['post', 'comentario', 'ser', 'remover', 'viol...",[''],[],NaN,Reddit,en,1.0,NaN,en,"['su', 'post', 'comentario', 'ha', 'sido', 're...",neg,neg,neg,neg
22157,22157,2021-08-06 17:19:38,NaN,NaN,Su post o comentario ha sido removido por viol...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-08-06,"['post', 'comentario', 'ser', 'remover', 'viol...",[''],[],NaN,Reddit,en,1.0,NaN,en,"['su', 'post', 'comentario', 'ha', 'sido', 're...",neg,neg,neg,neg
22158,22158,2021-08-11 23:03:45,NaN,NaN,It looks like you shared an AMP link. These sh...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-08-11,"['it', 'looks', 'like', 'you', 'shared', 'an',...",[''],['d:'],NaN,Reddit,en,2.0,NaN,en,"['look', 'like', 'share', 'amp', 'link', 'load...",pos,pos,neg,pos
22159,22159,2021-08-13 6:01:44,NaN,NaN,you can't troll the troll broh sorry.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-08-13,"['you', 'cannot', 'troll', 'the', 'troll', 'br...",[''],[],NaN,Reddit,en,3.0,NaN,en,"['cannot', 'troll', 'troll', 'broh', 'sorry']",neg,neg,neg,neg


In [ ]:
df = data_english[['sentiment_preprocessed_data_without_hashtags_textBlob', 'sentiment_preprocessed_data_without_hashtags_vader', 
                   'sentiment_preprocessed_data_without_hashtags_spacytextblob', 'sentiment_preprocessed_data_without_hashtags_flair']]

#finding the mode/majority
df['majority'] = df.mode(axis=1)[0]

#dropping columns
df = df.drop(['sentiment_text_emoji_emoticon_combined_textBlob', 'sentiment_text_emoji_emoticon_combined_vader', 
              'sentiment_text_emoji_emoticon_combined_spacytextblob', 'sentiment_text_emoji_emoticon_combined_flair'], axis = 1)

#concatinating two dataframe
df_sentiment = pd.concat([data_english, df], axis=1)
df_sentiment

#changing string to integer
score = []
for i in range(df_sentiment.shape[0]):
  text = df_sentiment['majority'].iloc[i]
  if text == 'pos':
    score.append('1')
  else:
    score.append('0')
df_sentiment['sentiment_score_text_emoji_emoticon_combined'] = score

In [ ]:
sentence = Sentence("I am happy")
classifier.predict(sentence)
comp = sentence.labels
print("POSITIVE" in str(comp))

True


In [ ]:
data_english = pd.read_csv("Corrected_Final_All_English_Flair_Column3-5.csv")

In [ ]:
data_english.shape

(5470, 39)

In [ ]:
data_english.head(20)

,Unnamed: 0,created_at,id_str,conversation_id_str,full_text,twitter_lang,favorited,retweeted,retweet_count,favorite_count,reply_count,quote_count,quoted_status_id_str,quoted_status_short_url,quoted_status_expand_url,user_id_str,user_name,user_full_name,user_verified,in_reply_to_status_id_str,in_reply_to_user_id_str,hashtags,mentions,urls,media,preprocessed_created_at,preprocessed_data,emoji_list,emoticons_list,filename,data_source,lang,score,langTb,lang_langdetect,preprocessed_data_without_hashtags,text_emoji_emoticon_combined,sentiment_text_emoji_emoticon_combined_flair,sentiment_full_text_without_handles_flair
0,16690,2021-04-23T07:14:36+00:00,1.390000e+18,1.390000e+18,📢Join the seminar aiming at supporting #1May #...,en,False,False,5.0,4.0,0.0,1.0,NaN,NaN,NaN,1.050000e+18,_EuropeanForum,European Forum,False,NaN,NaN,"#1May , #1maggio , #1Mayo","LeftHeinz , Yolanda_Diaz_ , VisentiniLuca , lu...",NaN,"[{""url"": ""https://pbs.twimg.com/media/EzpAFBfX...",23/04/21,"['join', 'seminar', 'aim', 'support', 'may', '...",['📢'],[],Mayo_ENGLISH_tweets_stweet.csv,Twitter,en,NaN,NaN,NaN,"['join', 'seminar', 'aim', 'support', 'trade',...","['join', 'seminar', 'aim', 'support', 'trade',...",[POSITIVE (0.9319)],[POSITIVE (0.6237)]
1,16691,2021-04-22T04:12:43+00:00,1.390000e+18,1.380000e+18,@julioacosta1701 @DiazCanelB @AlejandroGilF @r...,es,False,False,7.0,1.0,0.0,0.0,NaN,NaN,NaN,1.210000e+18,BauzaCuba,Alejandro Rodríguez Bauza,False,1.380000e+18,1.190000e+18,"#PrevenirPorLaVida , #PatriaOMuerte , #UnBlock...","julioacosta1701 , DiazCanelB , AlejandroGilF ,...",NaN,[],22/04/21,"['caliente', 'tweetazo', 'prevenirporlavida', ...",[],[],Mayo_ENGLISH_tweets_stweet.csv,Twitter,en,NaN,NaN,NaN,"['caliente', 'tweetazo']","['caliente', 'tweetazo']",[NEGATIVE (1.0)],[NEGATIVE (0.9998)]
2,16692,2021-04-22T04:12:00+00:00,1.390000e+18,1.380000e+18,@julioacosta1701 @AliRubioGlez @Vicente7397772...,es,False,False,7.0,1.0,0.0,0.0,NaN,NaN,NaN,1.210000e+18,BauzaCuba,Alejandro Rodríguez Bauza,False,1.380000e+18,1.190000e+18,"#PrevenirPorLaVida , #PatriaOMuerte , #UnBlock...","julioacosta1701 , AliRubioGlez , Vicente739777...",NaN,[],22/04/21,"['sumo', 'prevenirporlavida', 'patriaomuerte',...",[],[],Mayo_ENGLISH_tweets_stweet.csv,Twitter,en,NaN,NaN,NaN,['sumo'],['sumo'],[NEGATIVE (0.6003)],[NEGATIVE (0.9898)]
3,16693,2021-04-22T04:09:48+00:00,1.390000e+18,1.380000e+18,@AdrinMartnezGo1 @julioacosta1701 @DiazCanelB ...,es,False,False,6.0,0.0,0.0,0.0,NaN,NaN,NaN,1.210000e+18,BauzaCuba,Alejandro Rodríguez Bauza,False,1.390000e+18,1.140000e+18,"#PrevenirPorLaVida , #PatriaOMuerte , #UnBlock...","AdrinMartnezGo1 , julioacosta1701 , DiazCanelB...",NaN,[],22/04/21,"['hay', 'diocancancan', 'prevenirporlavida', '...",['😅😂🤣🤣'],[],Mayo_ENGLISH_tweets_stweet.csv,Twitter,en,NaN,NaN,NaN,"['hay', 'diocancancan']","['hay', 'diocancancan', '😅😂🤣🤣']",[NEGATIVE (0.9343)],[NEGATIVE (0.8989)]
4,16694,2021-04-22T03:32:26+00:00,1.390000e+18,1.380000e+18,@VilmaPrez14 @Laura53538740 @Granma_Digital @D...,es,False,False,5.0,1.0,0.0,1.0,NaN,NaN,NaN,1.210000e+18,BauzaCuba,Alejandro Rodríguez Bauza,False,1.380000e+18,1.100000e+18,"#PrevenirPorLaVida , #PatriaOMuerte , #UnBlock...","VilmaPrez14 , Laura53538740 , Granma_Digital ,...",NaN,[],22/04/21,"['con', 'mucha', 'conciencia', 'prevenirporlav...",[],[],Mayo_ENGLISH_tweets_stweet.csv,Twitter,en,NaN,NaN,NaN,"['con', 'mucha', 'conciencia']","['con', 'mucha', 'conciencia']",[POSITIVE (0.9088)],[POSITIVE (0.5925)]
5,16695,2021-04-20T14:58:57+00:00,1.380000e+18,1.380000e+18,"SOMOS CONTINUIDAD.\n#8voCongresoPCC, #CubaSoci...",es,False,False,0.0,1.0,0.0,0.0,NaN,NaN,NaN,1.090000e+18,EducacionDel,Educación Municipal Santa Cruz del Sur,False,NaN,NaN,"#8voCongresoPCC , #CubaSocialista , #SomosCont...","CamagueyDPE , elsa_ena , CubaMined , DayamiGal...",NaN,"[{""url"": ""https://pbs.twimg.com/media/EzbPO50X...",20/04/21,"['somos', 'continuidad', 'vocongresopcc', 'cub...",[],[],Mayo_ENGLISH_tweets_stweet.csv,Twitter,en,NaN,NaN,NaN,"['somos', 'continui

In [ ]:
data_english.columns

Index(['Unnamed: 0', 'created_at', 'id_str', 'conversation_id_str',
       'full_text', 'twitter_lang', 'favorited', 'retweeted', 'retweet_count',
       'favorite_count', 'reply_count', 'quote_count', 'quoted_status_id_str',
       'quoted_status_short_url', 'quoted_status_expand_url', 'user_id_str',
       'user_name', 'user_full_name', 'user_verified',
       'in_reply_to_status_id_str', 'in_reply_to_user_id_str', 'hashtags',
       'mentions', 'urls', 'media', 'preprocessed_created_at',
       'preprocessed_data', 'emoji_list', 'emoticons_list', 'filename',
       'data_source', 'lang', 'score', 'langTb', 'lang_langdetect',
       'preprocessed_data_without_hashtags', 'text_emoji_emoticon_combined',
       'sentiment_text_emoji_emoticon_combined_flair',
       'sentiment_full_text_without_handles_flair'],
      dtype='object')

In [ ]:
sentiment_score = []
for i in range(data_english.shape[0]):
  if "POSITIVE" in str(data_english['sentiment_text_emoji_emoticon_combined_flair'].iloc[i]):
    sentiment_score.append(1)
  else:
    sentiment_score.append(0)
data_english['sentiment_score'] = sentiment_score

In [ ]:
data_english.head(20)

,Unnamed: 0,created_at,id_str,conversation_id_str,full_text,twitter_lang,favorited,retweeted,retweet_count,favorite_count,reply_count,quote_count,quoted_status_id_str,quoted_status_short_url,quoted_status_expand_url,user_id_str,user_name,user_full_name,user_verified,in_reply_to_status_id_str,in_reply_to_user_id_str,hashtags,mentions,urls,media,preprocessed_created_at,preprocessed_data,emoji_list,emoticons_list,filename,data_source,lang,score,langTb,lang_langdetect,preprocessed_data_without_hashtags,text_emoji_emoticon_combined,sentiment_text_emoji_emoticon_combined_flair,sentiment_full_text_without_handles_flair,sentiment_score
0,16690,2021-04-23T07:14:36+00:00,1.390000e+18,1.390000e+18,📢Join the seminar aiming at supporting #1May #...,en,False,False,5.0,4.0,0.0,1.0,NaN,NaN,NaN,1.050000e+18,_EuropeanForum,European Forum,False,NaN,NaN,"#1May , #1maggio , #1Mayo","LeftHeinz , Yolanda_Diaz_ , VisentiniLuca , lu...",NaN,"[{""url"": ""https://pbs.twimg.com/media/EzpAFBfX...",23/04/21,"['join', 'seminar', 'aim', 'support', 'may', '...",['📢'],[],Mayo_ENGLISH_tweets_stweet.csv,Twitter,en,NaN,NaN,NaN,"['join', 'seminar', 'aim', 'support', 'trade',...","['join', 'seminar', 'aim', 'support', 'trade',...",[POSITIVE (0.9319)],[POSITIVE (0.6237)],1
1,16691,2021-04-22T04:12:43+00:00,1.390000e+18,1.380000e+18,@julioacosta1701 @DiazCanelB @AlejandroGilF @r...,es,False,False,7.0,1.0,0.0,0.0,NaN,NaN,NaN,1.210000e+18,BauzaCuba,Alejandro Rodríguez Bauza,False,1.380000e+18,1.190000e+18,"#PrevenirPorLaVida , #PatriaOMuerte , #UnBlock...","julioacosta1701 , DiazCanelB , AlejandroGilF ,...",NaN,[],22/04/21,"['caliente', 'tweetazo', 'prevenirporlavida', ...",[],[],Mayo_ENGLISH_tweets_stweet.csv,Twitter,en,NaN,NaN,NaN,"['caliente', 'tweetazo']","['caliente', 'tweetazo']",[NEGATIVE (1.0)],[NEGATIVE (0.9998)],0
2,16692,2021-04-22T04:12:00+00:00,1.390000e+18,1.380000e+18,@julioacosta1701 @AliRubioGlez @Vicente7397772...,es,False,False,7.0,1.0,0.0,0.0,NaN,NaN,NaN,1.210000e+18,BauzaCuba,Alejandro Rodríguez Bauza,False,1.380000e+18,1.190000e+18,"#PrevenirPorLaVida , #PatriaOMuerte , #UnBlock...","julioacosta1701 , AliRubioGlez , Vicente739777...",NaN,[],22/04/21,"['sumo', 'prevenirporlavida', 'patriaomuerte',...",[],[],Mayo_ENGLISH_tweets_stweet.csv,Twitter,en,NaN,NaN,NaN,['sumo'],['sumo'],[NEGATIVE (0.6003)],[NEGATIVE (0.9898)],0
3,16693,2021-04-22T04:09:48+00:00,1.390000e+18,1.380000e+18,@AdrinMartnezGo1 @julioacosta1701 @DiazCanelB ...,es,False,False,6.0,0.0,0.0,0.0,NaN,NaN,NaN,1.210000e+18,BauzaCuba,Alejandro Rodríguez Bauza,False,1.390000e+18,1.140000e+18,"#PrevenirPorLaVida , #PatriaOMuerte , #UnBlock...","AdrinMartnezGo1 , julioacosta1701 , DiazCanelB...",NaN,[],22/04/21,"['hay', 'diocancancan', 'prevenirporlavida', '...",['😅😂🤣🤣'],[],Mayo_ENGLISH_tweets_stweet.csv,Twitter,en,NaN,NaN,NaN,"['hay', 'diocancancan']","['hay', 'diocancancan', '😅😂🤣🤣']",[NEGATIVE (0.9343)],[NEGATIVE (0.8989)],0
4,16694,2021-04-22T03:32:26+00:00,1.390000e+18,1.380000e+18,@VilmaPrez14 @Laura53538740 @Granma_Digital @D...,es,False,False,5.0,1.0,0.0,1.0,NaN,NaN,NaN,1.210000e+18,BauzaCuba,Alejandro Rodríguez Bauza,False,1.380000e+18,1.100000e+18,"#PrevenirPorLaVida , #PatriaOMuerte , #UnBlock...","VilmaPrez14 , Laura53538740 , Granma_Digital ,...",NaN,[],22/04/21,"['con', 'mucha', 'conciencia', 'prevenirporlav...",[],[],Mayo_ENGLISH_tweets_stweet.csv,Twitter,en,NaN,NaN,NaN,"['con', 'mucha', 'conciencia']","['con', 'mucha', 'conciencia']",[POSITIVE (0.9088)],[POSITIVE (0.5925)],1
5,16695,2021-04-20T14:58:57+00:00,1.380000e+18,1.380000e+18,"SOMOS CONTINUIDAD.\n#8voCongresoPCC, #CubaSoci...",es,False,False,0.0,1.0,0.0,0.0,NaN,NaN,NaN,1.090000e+18,EducacionDel,Educación Municipal Santa Cruz del Sur,False,NaN,NaN,"#8voCongresoPCC , #CubaSocialista , #SomosCont...","CamagueyDPE , elsa_ena , CubaMined , DayamiGal...",NaN,"[{""url"": ""https://pbs.twimg.com/media/EzbPO50X...",20/04/21,"['somos', 'continuidad', 'vocongresopcc', 'cub...",[],[],Mayo_ENGLISH_tweets_stweet.csv,Twitter,en,NaN,Na

In [ ]:
data_english = data_english.drop(['sentiment_text_emoji_emoticon_combined_flair','sentiment_full_text_without_handles_flair','Unnamed: 0'],axis=1)

In [ ]:
data_english = data_english.drop(['text_emoji_emoticon_combined'], axis=1)

In [ ]:
data_english.shape

(5470, 36)

In [ ]:
data_english.to_csv("Final_all_english_sentiment.csv",index=False)
files.download("Final_all_english_sentiment.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
dict_count_positive = {}
dict_count_negative = {}

for i in range(data_english.shape[0]):
  score = data_english['sentiment_score'].iloc[i]
  x  = ast.literal_eval(data_english['preprocessed_data_without_hashtags'].iloc[i])
  for item in x:
    if score==1:
      if item not in dict_count_positive:
        dict_count_positive[item] = 1
      else:
        dict_count_positive[item]+=1
    else:
      if item not in dict_count_negative:
        dict_count_negative[item] = 1
      else:
        dict_count_negative[item]+=1
print(len(dict_count_positive))
print(len(dict_count_negative))

3417
6950


In [ ]:
sorted_dict_count_positive = dict(sorted(dict_count_positive.items(), key=operator.itemgetter(1),reverse=True))
sorted_dict_count_negative = dict(sorted(dict_count_negative.items(), key=operator.itemgetter(1),reverse=True))

In [ ]:
count=0
for key,value in sorted_dict_count_positive.items():
  print("{} : {}".format(key,value))
  count+=1
  if count==50:
    break

colombia : 173
help : 162
u : 140
people : 112
stamp : 111
right : 90
colombian : 89
por : 87
la : 78
favor : 76
police : 64
world : 59
country : 58
government : 51
de : 50
go : 50
amp : 48
day : 45
love : 45
cali : 45
well : 44
take : 44
happen : 43
solidarity : 43
need : 42
en : 41
please : 40
know : 39
protest : 38
like : 36
good : 35
gracias : 35
one : 35
make : 34
today : 34
human : 34
see : 33
thank : 33
kill : 32
calles : 31
national : 31
I : 30
fight : 29
share : 29
city : 28
time : 27
work : 27
stop : 27
support : 26
thanks : 26


In [ ]:
count=0
for key,value in sorted_dict_count_negative.items():
  print("{} : {}".format(key,value))
  count+=1
  if count==50:
    break

people : 821
colombia : 781
police : 618
government : 569
colombian : 444
protest : 434
tax : 366
u : 357
kill : 351
right : 339
help : 339
country : 331
go : 296
like : 282
need : 263
amp : 259
one : 246
please : 246
protester : 231
say : 221
get : 211
would : 209
reform : 202
take : 199
make : 193
want : 184
know : 175
I : 171
use : 163
day : 162
stop : 162
also : 160
money : 158
force : 158
even : 158
see : 154
violence : 152
many : 146
human : 146
thing : 144
time : 143
well : 141
pay : 137
state : 133
year : 130
happen : 129
shoot : 128
duque : 128
think : 127
cali : 122


In [ ]:
data_spanish = pd.read_csv("SentimentAnalysis_Spanish_Sentipy.csv")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (5,6,7,16,17,18,21,22,23,24,29,33) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
data_english.columns

Index(['created_at', 'id_str', 'conversation_id_str', 'full_text',
       'twitter_lang', 'favorited', 'retweeted', 'retweet_count',
       'favorite_count', 'reply_count', 'quote_count', 'quoted_status_id_str',
       'quoted_status_short_url', 'quoted_status_expand_url', 'user_id_str',
       'user_name', 'user_full_name', 'user_verified',
       'in_reply_to_status_id_str', 'in_reply_to_user_id_str', 'hashtags',
       'mentions', 'urls', 'media', 'preprocessed_created_at',
       'preprocessed_data', 'emoji_list', 'emoticons_list', 'filename',
       'data_source', 'lang', 'score', 'langTb', 'lang_langdetect',
       'preprocessed_data_without_hashtags', 'text_emoji_emoticon_combined',
       'sentiment_score'],
      dtype='object')

In [ ]:
data_spanish.columns

Index(['Unnamed: 0', 'created_at', 'id_str', 'conversation_id_str',
       'full_text', 'twitter_lang', 'favorited', 'retweeted', 'retweet_count',
       'favorite_count', 'reply_count', 'quote_count', 'quoted_status_id_str',
       'quoted_status_short_url', 'quoted_status_expand_url', 'user_id_str',
       'user_name', 'user_full_name', 'user_verified',
       'in_reply_to_status_id_str', 'in_reply_to_user_id_str', 'hashtags',
       'mentions', 'urls', 'media', 'preprocessed_created_at',
       'preprocessed_data', 'emoji_list', 'emoticons_list', 'filename',
       'data_source', 'lang', 'score', 'langTb', 'lang_langdetect',
       'preprocessed_data_without_hashtags', 'sentiment_full_text_sentipy',
       'sentiment_full_text_preprocessed_sentipy',
       'sentiment_preprocessed_data_without_hashtags_sentipy',
       'sentiment_emojis'],
      dtype='object')

In [ ]:
data_spanish.head()

,created_at,id_str,conversation_id_str,full_text,twitter_lang,favorited,retweeted,retweet_count,favorite_count,reply_count,quote_count,quoted_status_id_str,quoted_status_short_url,quoted_status_expand_url,user_id_str,user_name,user_full_name,user_verified,in_reply_to_status_id_str,in_reply_to_user_id_str,hashtags,mentions,urls,media,preprocessed_created_at,preprocessed_data,emoji_list,emoticons_list,filename,data_source,lang,score,langTb,lang_langdetect,preprocessed_data_without_hashtags,sentiment_score
0,2021-03-27T04:09:42+00:00,1.380000e+18,1.380000e+18,Hagamos otro por el uno de mayo que cazaremo...,es,False,False,0.0,1.0,0.0,0.0,NaN,NaN,NaN,9.094314e+08,carmeno1635,🌻🌺Carmen Ochoa...🌺🌻,False,1.380000e+18,1.290000e+18,#1mayo,NaN,NaN,"[{""url"": ""https://pbs.twimg.com/media/ExdUpC3X...",2021-03-27,"['hacer', 'mayo', 'cazar', 'fantasma', 'mayo']",[''],[':/'],Mayo_SPANISH_tweets_stweet.csv,Twitter,es,NaN,NaN,NaN,"['hacer', 'mayo', 'cazar', 'fantasma']",1
1,2021-03-22T21:12:09+00:00,1.370000e+18,1.370000e+18,Después de esperar con ancias el ahora esper...,es,False,False,1.0,4.0,0.0,0.0,NaN,NaN,NaN,1.260000e+18,olga73427503,olga Lidia 👄💄🌹,False,NaN,NaN,"#28F , #1Mayo , #15Septiembre","nayibbukele , waraujo64",NaN,[],2021-03-22,"['despues', 'esperar', 'ancia', 'ahora', 'espe...",['💙🤍💙'],[],Mayo_SPANISH_tweets_stweet.csv,Twitter,es,NaN,NaN,NaN,"['despues', 'esperar', 'ancia', 'ahora', 'espe...",1
2,2021-03-22T12:30:53+00:00,1.370000e+18,1.370000e+18,Espero que ésto llegue hasta oídos de la nueva...,es,False,False,0.0,1.0,1.0,0.0,NaN,NaN,NaN,1.350000e+18,Mr_DonNadie,Dios del Olimpo 🇸🇻⚖️ UES,False,NaN,NaN,"#1Mayo , #BecasUniversitarias",AsambleaSV,NaN,[],2021-03-22,"['esperar', 'llegar', 'oido', 'nuevo', 'inicia...",[''],[],Mayo_SPANISH_tweets_stweet.csv,Twitter,es,NaN,NaN,NaN,"['esperar', 'llegar', 'oido', 'nuevo', 'inicia...",0
3,2021-04-04T12:56:55+00:00,1.380000e+18,1.380000e+18,A menos de un mes del Urkullu teme perder el...,es,False,False,3.0,5.0,0.0,0.0,NaN,NaN,NaN,1.662306e+08,Antifaxismoa,Sare Antifaxista,False,NaN,NaN,"#1Mayo , #pandemia , #oasisvasco , #regimenjel...",NaN,NaN,[],2021-04-04,"['menos', 'mes', 'mayo', 'urkullu', 'temer', '...",[''],[],Mayo_SPANISH_tweets_stweet.csv,Twitter,es,NaN,NaN,NaN,"['menos', 'mes', 'urkullu', 'temer', 'perder',...",0
4,2021-04-03T20:14:57+00:00,1.380000e+18,1.380000e+18,La X Edición del Festival Internacional Un Pue...,es,False,False,1.0,3.0,0.0,1.0,NaN,NaN,NaN,1.320000e+18,Radiometrohaba1,Radio Metropolitana Cuba,False,NaN,NaN,"#LaHabana , #1Mayo , #MiHabanaEsMúsica , #Cuba...","AneliPupo , BenitezCordovi , LoureiroMilan , n...",https://t.co/caGq26zE1K,"[{""url"": ""https://pbs.twimg.com/media/EyE0s-AW...",2021-04-03,"['edicion', 'festival', 'internacional', 'puen...",[''],"[':/', ':/']",Mayo_SPANISH_tweets_stweet.csv,Twitter,es,NaN,NaN,NaN,"['edicion', 'festival', 'internacional', 'puen...",1


In [ ]:
list_1 = []
list_2 = []
list_3 = []
for i in range(data_spanish.shape[0]):
  list_1.append(int(data_spanish['sentiment_full_text_sentipy'].values[i] >= 0.5))
  list_2.append(int(data_spanish['sentiment_full_text_preprocessed_sentipy'].values[i] >= 0.5))
  list_3.append(int(data_spanish['sentiment_preprocessed_data_without_hashtags_sentipy'].values[i] >= 0.5))

list_of_tuples = list(zip(list_1, list_2, list_3))
df = pd.DataFrame(list_of_tuples,columns = ['l1', 'l2', 'l3'])


,l1,l2,l3
0,1,1,1
1,1,1,1
2,0,0,0
3,0,0,0
4,1,1,1
...,...,...,...
16685,0,0,0
16686,0,0,0
16687,0,0,0
16688,0,0,0


In [ ]:
df['majority'] = df.mode(axis=1)[0]
df

,l1,l2,l3,majority
0,1,1,1,1
1,1,1,1,1
2,0,0,0,0
3,0,0,0,0
4,1,1,1,1
...,...,...,...,...
16685,0,0,0,0
16686,0,0,0,0
16687,0,0,0,0
16688,0,0,0,0


In [ ]:
data_spanish['sentiment_score'] = df['majority']

In [ ]:
data_spanish.shape

(16690, 41)

In [ ]:
data_spanish = data_spanish.drop(['sentiment_full_text_sentipy', 'sentiment_full_text_preprocessed_sentipy', 
                   'sentiment_preprocessed_data_without_hashtags_sentipy', 'sentiment_emojis', 'Unnamed: 0'],axis=1)

In [ ]:
print(data_english.shape)
print(data_spanish.shape)

(5470, 36)
(16690, 36)


In [ ]:
data = pd.concat([data_spanish, data_english], axis=0)

In [ ]:
data.shape

(22160, 36)

In [ ]:
data.tail()

,created_at,id_str,conversation_id_str,full_text,twitter_lang,favorited,retweeted,retweet_count,favorite_count,reply_count,quote_count,quoted_status_id_str,quoted_status_short_url,quoted_status_expand_url,user_id_str,user_name,user_full_name,user_verified,in_reply_to_status_id_str,in_reply_to_user_id_str,hashtags,mentions,urls,media,preprocessed_created_at,preprocessed_data,emoji_list,emoticons_list,filename,data_source,lang,score,langTb,lang_langdetect,preprocessed_data_without_hashtags,sentiment_score
5465,03/08/21 12:01,NaN,NaN,It looks like you shared an AMP link. These sh...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,03/08/21,"['it', 'looks', 'like', 'you', 'shared', 'an',...",[''],['d:'],NaN,Reddit,en,2.0,NaN,en,"['look', 'like', 'share', 'amp', 'link', 'load...",0
5466,05/08/21 22:02,NaN,NaN,Su post o comentario ha sido removido por viol...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,05/08/21,"['post', 'comentario', 'ser', 'remover', 'viol...",[''],[],NaN,Reddit,en,1.0,NaN,en,"['su', 'post', 'comentario', 'ha', 'sido', 're...",1
5467,06/08/21 17:19,NaN,NaN,Su post o comentario ha sido removido por viol...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,06/08/21,"['post', 'comentario', 'ser', 'remover', 'viol...",[''],[],NaN,Reddit,en,1.0,NaN,en,"['su', 'post', 'comentario', 'ha', 'sido', 're...",1
5468,11/08/21 23:03,NaN,NaN,It looks like you shared an AMP link. These sh...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11/08/21,"['it', 'looks', 'like', 'you', 'shared', 'an',...",[''],['d:'],NaN,Reddit,en,2.0,NaN,en,"['look', 'like', 'share', 'amp', 'link', 'load...",0
5469,13/08/21 6:01,NaN,NaN,you can't troll the troll broh sorry.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13/08/21,"['you', 'cannot', 'troll', 'the', 'troll', 'br...",[''],[],NaN,Reddit,en,3.0,NaN,en,"['cannot', 'troll', 'troll', 'broh', 'sorry']",0


In [ ]:
data_1 = pd.read_csv("Corrected_Final_All.csv")

In [ ]:
data_1.shape

(22160, 36)

In [ ]:
list_score = []
for item in data['sentiment_score']:
  list_score.append(item)
data_1['sentiment_score'] = list_score

In [ ]:
data_1.tail()

,Unnamed: 0,created_at,id_str,conversation_id_str,full_text,twitter_lang,favorited,retweeted,retweet_count,favorite_count,reply_count,quote_count,quoted_status_id_str,quoted_status_short_url,quoted_status_expand_url,user_id_str,user_name,user_full_name,user_verified,in_reply_to_status_id_str,in_reply_to_user_id_str,hashtags,mentions,urls,media,preprocessed_created_at,preprocessed_data,emoji_list,emoticons_list,filename,data_source,lang,score,langTb,lang_langdetect,preprocessed_data_without_hashtags,sentiment_score
22155,22155,2021-08-03 12:01:04,NaN,NaN,It looks like you shared an AMP link. These sh...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-08-03,"['it', 'looks', 'like', 'you', 'shared', 'an',...",[''],['d:'],NaN,Reddit,en,2.0,NaN,en,"['look', 'like', 'share', 'amp', 'link', 'load...",0
22156,22156,2021-08-05 22:02:30,NaN,NaN,Su post o comentario ha sido removido por viol...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-08-05,"['post', 'comentario', 'ser', 'remover', 'viol...",[''],[],NaN,Reddit,en,1.0,NaN,en,"['su', 'post', 'comentario', 'ha', 'sido', 're...",1
22157,22157,2021-08-06 17:19:38,NaN,NaN,Su post o comentario ha sido removido por viol...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-08-06,"['post', 'comentario', 'ser', 'remover', 'viol...",[''],[],NaN,Reddit,en,1.0,NaN,en,"['su', 'post', 'comentario', 'ha', 'sido', 're...",1
22158,22158,2021-08-11 23:03:45,NaN,NaN,It looks like you shared an AMP link. These sh...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-08-11,"['it', 'looks', 'like', 'you', 'shared', 'an',...",[''],['d:'],NaN,Reddit,en,2.0,NaN,en,"['look', 'like', 'share', 'amp', 'link', 'load...",0
22159,22159,2021-08-13 6:01:44,NaN,NaN,you can't troll the troll broh sorry.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-08-13,"['you', 'cannot', 'troll', 'the', 'troll', 'br...",[''],[],NaN,Reddit,en,3.0,NaN,en,"['cannot', 'troll', 'troll', 'broh', 'sorry']",0


In [ ]:
data_1 = data_1.drop(['Unnamed: 0'], axis=1)

In [ ]:
data_1.to_csv("Corrected_final_all_with_sentiment.csv")
files.download("Corrected_final_all_with_sentiment.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
dict_count_positive_s = {}
dict_count_negative_s = {}

for i in range(data_spanish.shape[0]):
  score = data_spanish['sentiment_score'].iloc[i]
  x  = ast.literal_eval(data_spanish['preprocessed_data_without_hashtags'].iloc[i])
  for item in x:
    if score==1:
      if item not in dict_count_positive_s:
        dict_count_positive_s[item] = 1
      else:
        dict_count_positive_s[item]+=1
    else:
      if item not in dict_count_negative_s:
        dict_count_negative_s[item] = 1
      else:
        dict_count_negative_s[item]+=1
print(len(dict_count_positive_s))
print(len(dict_count_negative_s))

6217
16192


In [ ]:
sorted_dict_count_positive_s = dict(sorted(dict_count_positive_s.items(), key=operator.itemgetter(1),reverse=True))
sorted_dict_count_negative_s = dict(sorted(dict_count_negative_s.items(), key=operator.itemgetter(1),reverse=True))

In [ ]:
count=0
for key,value in sorted_dict_count_positive_s.items():
  print("{} : {}".format(key,value))
  count+=1
  if count==50:
    break

mas : 348
hacer : 282
si : 259
pais : 251
colombio : 240
ir : 232
ser : 219
poder : 190
seguir : 177
ver : 171
decir : 169
él : 166
dia : 165
estar : 159
pueblo : 154
colombiano : 138
colombia : 136
calle : 132
solo : 124
buen : 122
gente : 120
hoy : 111
cali : 110
paro : 108
bien : 106
gobierno : 105
vida : 102
asi : 98
primero : 97
mejor : 97
nacional : 96
resistencia : 96
querer : 95
dar : 93
fuerza : 92
lucha : 90
joven : 87
deber : 86
momento : 85
vivo : 83
parte : 82
tú : 82
persona : 81
gracias : 79
todo : 79
pasar : 78
protesta : 77
tambien : 76
salir : 75
nuevo : 74


In [ ]:
count=0
for key,value in sorted_dict_count_negative_s.items():
  print("{} : {}".format(key,value))
  count+=1
  if count==50:
    break

mas : 2021
si : 1720
hacer : 1711
poder : 1363
ir : 1293
él : 1287
pais : 1177
ser : 1167
gobierno : 1149
colombio : 1072
decir : 1031
estar : 1000
paro : 926
pueblo : 914
ver : 852
seguir : 848
policia : 844
querer : 822
solo : 813
colombiano : 750
dar : 722
gente : 711
protesta : 583
derecho : 564
dia : 562
mismo : 551
persona : 548
salir : 545
dejar : 537
tú : 536
asi : 529
deber : 527
reforma : 518
joven : 496
colombia : 485
creer : 482
matar : 481
hoy : 480
tanto : 469
saber : 461
duque : 459
pasar : 454
vida : 453
social : 448
cali : 423
tambien : 418
todo : 414
ahora : 412
bien : 394
politico : 388
